#Code

In [ ]:
# @title Crewai With Groq
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and pydevd_plugins packages from being imported.
import sys
sys.modules.pop('PIL', None)




  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.3 MB/s eta 0:00:00
---
Name: crewai
Version: 0.55.2
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: https://crewai.com
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, auth0-python, click, embedchain, instructor, json-repair, jsonref, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 
---
Name: crewai-tools
Version: 0.12.0
Summary: Set of tools for the crewAI framework
Home-page: http

AttributeError: module 'PIL' has no attribute 'Image'

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere



In [ ]:
!pip install --upgrade Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 30.7 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [ ]:
# @title 🔑 Get API Key from Colab Secret by running this cell

from google.colab import userdata
import os
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from langchain_groq import ChatGroq  # Use Groq's API





In [ ]:
# Retrieve your Groq API key from Colab secret
groq_api_key = userdata.get('GROQ_API_KEY')

if groq_api_key:
    os.environ["GROQ_API_KEY"] = groq_api_key
else:
    raise ValueError("No Groq API Key found in Colab secrets. Please add it using Google Colab secret management.")

# Check if the 'output-files' directory exists, and create it if it doesn't
if not os.path.exists('output-files'):
    os.makedirs('output-files')

In [1]:
# @title 🕵🏻 Define your agents

# Agent 1: Get Stock Data
agent_1 = Agent(
    role=dedent((
        """
        Stock Data Fetcher: Retrieves the latest stock data from APIs like Yahoo Finance, Alpha Vantage, etc.
        """)),
    backstory=dedent((
        """
        I am responsible for fetching stock price data for further analysis.
        """)),
    goal=dedent((
        """
        Fetch stock data and deliver it to the next agent for analysis.
        """)),
    allow_delegation=True,
    verbose=True,
    max_iter=3,
    max_rpm=3,
    llm=ChatGroq(temperature=0.8, model_name="mixtral-8x7b-32768")
)

# Agent 2: Analyze Stock Data
agent_2 = Agent(
    role=dedent((
        """
        Stock Analyst: Performs a detailed analysis on the stock data (e.g., moving averages, RSI, trends).
        """)),
    backstory=dedent((
        """
        I am responsible for analyzing stock performance using technical and fundamental indicators.
        """)),
    goal=dedent((
        """
        Analyze the stock data and provide insights or recommendations based on trends and patterns.
        """)),
    allow_delegation=True,
    verbose=True,
    max_iter=3,
    max_rpm=3,
    llm=ChatGroq(temperature=0.8, model_name="llama3-70b-8192")
)

# Agent 3: Search for News about the Stock
agent_3 = Agent(
    role=dedent((
        """
        Stock News Fetcher: Searches for relevant financial news regarding the stock.
        """)),
    backstory=dedent((
        """
        I gather news about the stock to help make better-informed decisions and provide context to the analysis.
        """)),
    goal=dedent((
        """
        Search for the latest news related to the stock to add context to the report.
        """)),
    allow_delegation=True,
    verbose=True,
    max_iter=3,
    max_rpm=3,
    llm=ChatGroq(temperature=0.6, model_name="mixtral-8x7b-32768")
)

# Agent 4: Publish Report
agent_4 = Agent(
    role=dedent((
        """
        Stock Report Publisher: Gathers all the analyzed data and news and creates a final report.
        """)),
    backstory=dedent((
        """
        I compile the analysis and news data into a comprehensive report for publishing or sharing.
        """)),
    goal=dedent((
        """
        Publish a report that includes stock data, analysis, and recent news.
        """)),
    allow_delegation=True,
    verbose=True,
    max_iter=3,
    max_rpm=3,
    llm=ChatGroq(temperature=0.8, model_name="mixtral-8x7b-32768")
)



NameError: name 'Agent' is not defined

In [ ]:
# @title 📝 Define your tasks

# Task 1: Get stock data
task_1 = Task(
    description=dedent((
        """
        Retrieve stock data for the specified ticker symbol.
        ---
        Ticker: "{ticker}"
        """)),
    expected_output=dedent((
        """
        A detailed report of the stock's historical data, current price, and key financial metrics.
        """)),
    agent=agent_1,
)

# Task 2: Analyze stock data
task_2 = Task(
    description=dedent((
        """
        Analyze the stock data to identify trends and make predictions.
        ---
        Ticker: "{ticker}"
        """)),
    expected_output=dedent((
        """
        An analysis report detailing the stock's trends, key technical indicators (e.g., moving averages, RSI), and any recommendations.
        """)),
    agent=agent_2,
    context=[task_1],  # Uses the output of Task 1 (stock data)
)

# Task 3: Search for news about the stock
task_3 = Task(
    description=dedent((
        """
        Find the latest news articles and financial reports about the stock.
        ---
        Ticker: "{ticker}"
        """)),
    expected_output=dedent((
        """
        A collection of relevant news articles or reports about the stock.
        """)),
    agent=agent_3,
    context=[task_1, task_2],  # Uses output from Task 1 and Task 2
)

# Task 4: Publish the report
task_4 = Task(
    description=dedent((
        """
        Create a comprehensive stock report including data, analysis, and news.
        ---
        Ticker: "{ticker}"
        """)),
    expected_output=dedent((
        """
        A detailed stock report ready to be published or shared.
        """)),
    agent=agent_4,
    context=[task_1, task_2, task_3],  # Uses output from Task 1, 2, and 3
)



In [ ]:
# @title ⌨️ Define any variables you have and input them

ticker = input("Enter the stock ticker symbol to analyze (e.g., AAPL, TSLA, MSFT): ")

# @title 🚀 Get your crew to work!

def main():
    # Instantiate your crew with a hierarchical process
    crew = Crew(
        agents=[agent_1, agent_2, agent_3, agent_4],
        tasks=[task_1, task_2, task_3, task_4],
        verbose=True,  # You can set it to True or False
        process=Process.hierarchical,  # Define the process flow as hierarchical
        manager_llm=ChatGroq(temperature=0.8, model_name="mixtral-8x7b-32768")  # Groq as manager LLM
    )

    inputs = {
        "ticker": ticker
    }

    result = crew.kickoff(inputs=inputs)
    print("\n\n########################")
    print("## Here is your custom crew run result:")
    print("########################\n")
    print(result)

    return result

if __name__ == "__main__":
    result = main()



Enter the stock ticker symbol to analyze (e.g., AAPL, TSLA, MSFT): AAPL
 [2024-09-08 16:41:39][DEBUG]: == Working Agent: Crew Manager
 [2024-09-08 16:41:39][INFO]: == Starting Task: 
Retrieve stock data for the specified ticker symbol.
---
Ticker: "AAPL"



> Entering new CrewAgentExecutor chain...
To retrieve the stock data for the specified ticker symbol "AAPL", I need to delegate the task of fetching the historical data, current price, and key financial metrics to the Stock Data Fetcher coworker. I'll make sure to provide them with all the necessary context and parameters.

Action: Delegate work to coworker
Action Input: {
  'task': 'Fetch historical data, current price, and key financial metrics',
  'context': 'Ticker symbol is AAPL (Apple Inc.). The historical data should cover at least 1 year, preferably 5 years. Key financial metrics include but are not limited to: market capitalization, P/E ratio, EPS, and dividend yield.',
  'coworker': 'Stock Data Fetcher'
}
 


Error executi

Here’s a concise summary of the conversation flow based on your task:

1. **Stock Ticker Entry (AAPL)**: The system starts by requesting data for "AAPL".
   
2. **Delegate to Stock Data Fetcher**: Task to retrieve stock data (historical, financial metrics) encounters errors repeatedly due to tool misconfiguration.

3. **Delegate to Stock Analyst**: Multiple failed attempts to calculate technical indicators like moving averages and RSI.

4. **Final Analysis Plan**: Despite tool issues, a hypothetical analysis plan was presented (moving averages, RSI, trends).

5. **News and Reports Request**: Attempts to delegate the task of retrieving news articles and financial reports, also repeatedly failed due to the same tool issue.

6. **Stock Report Summary**: A comprehensive stock report creation plan was outlined but could not be executed due to tool malfunction.

7. **Conclusion**: Unable to complete tasks due to persistent tool recognition errors.

In [ ]:
# @title 🖥️ Display the results of your crew as markdown

from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))

I regret to inform you that I am unable to complete the task of creating a comprehensive stock report for "AAPL" due to an issue with the provided tools. If the tools were functioning properly, I would gather the latest stock data, calculate technical indicators, find relevant news articles and financial reports, and write a report summarizing my findings and providing recommendations. However, I am currently unable to use the tools to perform these tasks.

#lisence

https://github.com/alexfazio/crewAI-quickstart/blob/main/crewai_hierarchical_quickstart.ipynb

MIT License

Copyright (c) 2024 Alex Fazio

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Modification notes: The original cookbook shows basic setting structures for creating agents tasks and orchestrating the task, without specifycing any details. This version shows multiple modifications which is not present in the original file, and subject to MIT lisence, author Steven(youtube channel LearnByDoing).

Tool settings(when agents call tools, without engaging code agent)
Detailed agent settings
Detailed tasks settings
Objective is to analyze a stock performance, gather information, draft final reports

MIT License

Copyright (c) 2024 Steven(LearnByDoing Youtube Channel)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.